# Preconfiguration

In [1]:
import time as t

##### Pre-Configuration of Spark

function sparknotebook()
{
export SPARK_LOCAL_IP="192.168.10.1"
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook --no-browser --NotebookApp.token='' --allow-root --NotebookApp.ip='*' --NotebookApp.port=8880"
nohup $SPARK_HOME/bin/pyspark --master spark://192.168.10.1:7077
}

*** NO ANY CRONTAB JOB ***

In [2]:
! rm -rf metastore_db/

In [3]:
! pip install pandas

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from pyspark.sql.functions import *

In [5]:
spark.stop()

In [6]:
spark = SparkSession \
.builder \
.appName("Prediction of Loan Payment") \
.master("spark://192.168.10.1:7077") \
.config("spark.executor.cores","1")\
.getOrCreate();

In [7]:
spark

In [8]:
raw_LendingClubWeb_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('/root/notebook/data/LoanStats_web.csv')

##### Get row count

In [9]:
raw_LendingClubWeb_df.count()

1432439

##### Get column count

In [10]:
len(raw_LendingClubWeb_df.columns)

144

##### Get Data Type

In [11]:
raw_LendingClubWeb_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

##### Get Freq. with Various Data Group for Detection of Some Outliers and other Noises.

In [12]:
raw_LendingClubWeb_df.select(['purpose']).groupBy('purpose').count().show()

+------------------+------+
|           purpose| count|
+------------------+------+
|           wedding|     7|
|             other| 93997|
|    small_business| 15032|
|debt_consolidation|791531|
|       credit_card|335091|
|            moving|  9733|
|          vacation| 10579|
|  renewable_energy|   910|
|             house|  9893|
|               car| 15288|
|    major_purchase| 32649|
|           medical| 18807|
|  home_improvement| 98922|
+------------------+------+



In [13]:
raw_LendingClubWeb_df.groupBy('purpose')\
.agg(avg(col("loan_amnt")),sum(col("loan_amnt"))).show()

+------------------+------------------+---------------+
|           purpose|    avg(loan_amnt)| sum(loan_amnt)|
+------------------+------------------+---------------+
|           wedding|11967.857142857143|        83775.0|
|             other|10855.311339723608|    1.0203667E9|
|    small_business|17023.697778073445|   2.55900225E8|
|debt_consolidation|16461.308685319967|1.3029636125E10|
|       credit_card|15365.308005884968|  5.148776425E9|
|            moving| 8649.018802013768|      8.41809E7|
|          vacation| 6489.663484261272|     6.865415E7|
|  renewable_energy|11595.631868131868|    1.0552025E7|
|             house| 16221.37875265339|     1.604781E8|
|               car| 9686.968537414967|   1.48094375E8|
|    major_purchase|13169.705810285153|   4.29977725E8|
|           medical|  9730.98314457383|     1.830106E8|
|  home_improvement|14935.395311457512|  1.477439175E9|
+------------------+------------------+---------------+



In [14]:
raw_LendingClubWeb_df.select(['home_ownership']).groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN|170888|
|          RENT|556960|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



In [15]:
raw_LendingClubWeb_df.select(['grade']).groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F| 19480|
|    E| 68353|
|    B|424110|
|    D|191136|
|    C|421101|
|    A|302988|
|    G|  5271|
+-----+------+



In [16]:
raw_LendingClubWeb_df.select(['emp_title']).groupBy('emp_title').count().show()

+--------------------+-----+
|           emp_title|count|
+--------------------+-----+
|Sr. VP of Busines...|    5|
|Journeyman Electr...|  146|
|           Physician| 1337|
|Applications prog...|    4|
|                 CSR|  694|
|   Project Executive|   38|
|       Police Office|   16|
|Physical therapis...|   54|
|   Principal planner|    4|
|           Scientist|  317|
|        Store Leader|   46|
|            Sale rep|    5|
|  Custodial Engineer|    8|
|   Sanitation worker|   55|
|           machinist|  429|
|Implementation Co...|   87|
|                 CRS|   13|
|Asst commonwealth...|    4|
|Digital Marketing...|   69|
|Lead Radiologic T...|    4|
+--------------------+-----+
only showing top 20 rows



In [17]:
raw_LendingClubWeb_df.select(['emp_length']).groupBy('emp_length').count().show()

+----------+------+
|emp_length| count|
+----------+------+
|   5 years| 88686|
|   9 years| 46393|
|    1 year| 95926|
|       n/a|108470|
|   2 years|130250|
|   7 years| 48965|
|   8 years| 51222|
|   4 years| 86767|
|   6 years| 62261|
|   3 years|116802|
| 10+ years|478304|
|  < 1 year|118393|
+----------+------+



In [18]:
raw_LendingClubWeb_df.select(['loan_status']).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|583755|
|           Default|  1419|
|   In Grace Period|  6136|
|       Charged Off|152998|
|Late (31-120 days)| 19455|
|           Current|663690|
| Late (16-30 days)|  4986|
+------------------+------+



In [19]:
raw_LendingClubWeb_df.select(['term']).groupBy('term').count().show()

+----------+-------+
|      term|  count|
+----------+-------+
| 36 months|1036234|
| 60 months| 396205|
+----------+-------+



In [20]:
raw_LendingClubWeb_df.select(['pub_rec']).groupBy('pub_rec').count().show()

+-------+-------+
|pub_rec|  count|
+-------+-------+
|      7|    302|
|     15|      4|
|     11|     39|
|      3|   6827|
|      8|    111|
|     28|      9|
|     22|      1|
|     52|      1|
|     16|     14|
|      0|1196699|
|     47|      1|
|      5|   1300|
|     31|      1|
|     61|      5|
|     17|      2|
|     46|      1|
|      6|    629|
|     19|     16|
|     25|      1|
|      9|     99|
+-------+-------+
only showing top 20 rows



In [21]:
raw_LendingClubWeb_df.select(['inq_last_6mths']).groupBy('inq_last_6mths').count().show()

+--------------+------+
|inq_last_6mths| count|
+--------------+------+
|             3| 31289|
|             0|925659|
|          null|     1|
|             5|  2400|
|             1|356887|
|             4|  7274|
|             2|108929|
+--------------+------+



In [22]:
raw_LendingClubWeb_df.select(['settlement_status']).groupBy('settlement_status').count().show()

+-----------------+-------+
|settlement_status|  count|
+-----------------+-------+
|             null|1407028|
|         COMPLETE|   8606|
|           BROKEN|   4116|
|           ACTIVE|  12689|
+-----------------+-------+



##### Get data that contain ONLY Fully Paid and Charged Off

In [23]:
loanPayment_df = raw_LendingClubWeb_df\
.filter((col('loan_status') == 'Fully Paid') | ((col('loan_status') =='Charged Off')))

In [24]:
loanPayment_df.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|583755|
|Charged Off|152998|
+-----------+------+



In [25]:
loanPayment_df.count()

736753

##### Get data that contain ONLY related attributes with Business Prespective.

In [26]:
business_df = loanPayment_df.select('annual_inc'\
                                           ,'bc_util'\
                                           ,'inq_fi'\
                                           ,'inq_last_12m'\
                                           ,'home_ownership'\
                                           ,'purpose'\
                                           ,'emp_length'\
                                           ,'revol_bal'\
                                           ,'dti'\
                                           ,'delinq_2yrs'\
                                           ,'pub_rec_bankruptcies'\
                                           ,'pub_rec'\
                                           ,'open_rv_24m'\
                                           ,'mort_acc'\
                                           ,'num_actv_bc_tl'\
                                           ,'num_actv_rev_tl'\
                                           ,'num_il_tl'\
                                           ,'num_tl_90g_dpd_24m'\
                                           ,'int_rate'\
                                           ,'inq_last_6mths'\
                                           ,'term'\
                                           ,'installment'\
                                           ,'total_rev_hi_lim'\
                                           ,'total_bal_il'\
                                           ,'total_bal_ex_mort'\
                                           ,'total_acc'\
                                           ,'tot_cur_bal'\
                                           ,'loan_amnt'\
                                           ,'loan_status'\
                                           ,'verification_status'\
                                           ,'collections_12_mths_ex_med'\
                                           ,'chargeoff_within_12_mths'\
                                           ,'il_util'\
                                           ,'last_pymnt_amnt'\
                                           #,'last_pymnt_d'\
                                           ,'out_prncp_inv'\
                                           ,'out_prncp'\
                                           ,'total_pymnt_inv'\
                                           ,'total_pymnt'\
                                           ,'grade')
                                           #,'sec_app_collections_12_mths_ex_med'\
                                           #,'sec_app_chargeoff_within_12_mths'\
                                           #,'settlement_term'\
                                           #,'settlement_amount'\
                                           #,'settlement_status'

In [27]:
business_df.count()

736753

In [28]:
len(business_df.columns)

39

##### Get Data Profiling

In [29]:
import pandas as pd
business_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,736753,79498.51232281378,78094.80304273874,0,99999.84
bc_util,727863,55.10997234369625,28.90306464396113,0,99.9
inq_fi,736697,1.058429720767154,1.5524238231939935,0,9
inq_last_12m,736696,2.226637581851944,2.492919247096462,0,9
home_ownership,736753,None,None,ANY,RENT
purpose,736753,None,None,car,wedding
emp_length,736753,None,None,1 year,n/a
revol_bal,736753,16101.816257280256,22819.366258659797,0,99991
dti,736281,18.711086881231356,13.79426027607045,-1,999


### สรุปการดำเนินงานของ Data Understanding: เลือกเฉพาะบาง Attributes และ Row ที่เป็น Charge Off/Fully Charge

### 3) Data Preparation

##### Get Data with No Null

In [30]:
no_null_df = business_df.dropna(how='any')

In [31]:
#no_null_df.count()

##### Get Data with No "wedding"

In [32]:
from pyspark.sql.functions import *

In [33]:
no_wedding_df = no_null_df.filter(col('purpose') != 'wedding')

In [34]:
no_wedding_df.select(['purpose']).groupBy('purpose').count().show()

+------------------+------+
|           purpose| count|
+------------------+------+
|             other| 41383|
|    small_business|  6552|
|debt_consolidation|360885|
|       credit_card|130378|
|            moving|  4877|
|          vacation|  5049|
|  renewable_energy|   445|
|             house|  3758|
|               car|  6838|
|    major_purchase| 14407|
|           medical|  8449|
|  home_improvement| 44688|
+------------------+------+



In [35]:
no_wedding_df.count()

627709

### นับจำนวนข้อมูลหลัง Cleansing ค่า 'Null'

In [36]:
fitmem_no_null_df = no_wedding_df.repartition(60)

In [37]:
cached_no_null_df = fitmem_no_null_df.cache()

In [38]:
cached_no_null_df.count()

627709

### Remove '%' และ Extract Month

In [39]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove '%' 

In [40]:
def f_removepercent(origin):
    return origin.rstrip('%')

In [41]:
removepercent = udf(lambda x: f_removepercent(x),StringType())

### Remove 'Wedding'

In [42]:
#def f_wedding(origin):
    #return origin.rstrip('wedding')

In [43]:
#removewedding = udf(lambda x: f_wedding(x),StringType())

### Extract Month

In [44]:
def f_exrtractmonth(origin):
    return origin.split('-')[0]

In [45]:
exrtractmonth = udf(lambda x: f_exrtractmonth(x),StringType())

### Replace'n/a' ใน Field 'emp_length' ด้วย 'No employ'

In [46]:
def python_treatNA(origin):
    if origin == 'n/a':
        new = 'NotEmployed'
    else:
        new = origin
    return new

In [47]:
treatNA = udf(lambda x: python_treatNA(x),StringType())

### เปลี่ยนแปลง Type ของข้อมูล

In [48]:
from pyspark.sql.functions import col

In [49]:
crunched_df = cached_no_null_df.\
withColumn('emp_length',treatNA(cached_no_null_df['emp_length'])).\
withColumn('int_rate',removepercent(cached_no_null_df['int_rate']).cast(DoubleType())).\
withColumn('dti',cached_no_null_df['dti'].cast(DoubleType())).\
withColumn('revol_bal',cached_no_null_df['revol_bal'].cast(DoubleType())).\
withColumn('pub_rec',cached_no_null_df['pub_rec'].cast(DoubleType())).\
withColumn('total_bal_il',cached_no_null_df['total_bal_il'].cast(DoubleType())).\
withColumn('tot_cur_bal',cached_no_null_df['tot_cur_bal'].cast(DoubleType())).\
withColumn('total_acc',cached_no_null_df['total_acc'].cast(DoubleType())).\
withColumn('total_bal_ex_mort',cached_no_null_df['total_bal_ex_mort'].cast(DoubleType())).\
withColumn('total_rev_hi_lim',cached_no_null_df['total_rev_hi_lim'].cast(DoubleType())).\
withColumn('num_actv_rev_tl',cached_no_null_df['num_actv_rev_tl'].cast(DoubleType())).\
withColumn('num_actv_bc_tl',cached_no_null_df['num_actv_bc_tl'].cast(DoubleType())).\
withColumn('num_il_tl',cached_no_null_df['num_il_tl'].cast(DoubleType())).\
withColumn('pub_rec_bankruptcies',cached_no_null_df['pub_rec_bankruptcies'].cast(DoubleType())).\
withColumn('delinq_2yrs',cached_no_null_df['delinq_2yrs'].cast(DoubleType())).\
withColumn('open_rv_24m',cached_no_null_df['open_rv_24m'].cast(DoubleType())).\
withColumn('num_tl_90g_dpd_24m',cached_no_null_df['num_tl_90g_dpd_24m'].cast(DoubleType())).\
withColumn('inq_last_6mths',cached_no_null_df['inq_last_6mths'].cast(DoubleType())).\
withColumn('bc_util',cached_no_null_df['bc_util'].cast(DoubleType())).\
withColumn('mort_acc',cached_no_null_df['mort_acc'].cast(DoubleType())).\
withColumn('inq_fi',cached_no_null_df['inq_fi'].cast(DoubleType())).\
withColumn('last_pymnt_amnt',cached_no_null_df['last_pymnt_amnt'].cast(DoubleType())).\
withColumn('out_prncp_inv',cached_no_null_df['out_prncp_inv'].cast(DoubleType())).\
withColumn('out_prncp',cached_no_null_df['out_prncp'].cast(DoubleType())).\
withColumn('total_pymnt_inv',cached_no_null_df['total_pymnt_inv'].cast(DoubleType())).\
withColumn('total_pymnt',cached_no_null_df['total_pymnt'].cast(DoubleType())).\
withColumn('il_util',cached_no_null_df['il_util'].cast(DoubleType())).\
withColumn('chargeoff_within_12_mths',cached_no_null_df['chargeoff_within_12_mths'].cast(DoubleType())).\
withColumn('collections_12_mths_ex_med',cached_no_null_df['collections_12_mths_ex_med'].cast(DoubleType())).\
withColumn('loan_amnt',cached_no_null_df['loan_amnt'].cast(DoubleType())).\
withColumn('inq_last_12m',cached_no_null_df['inq_last_12m'].cast(DoubleType())).\
withColumn('installment',cached_no_null_df['installment'].cast(DoubleType())).\
withColumn('annual_inc',cached_no_null_df['annual_inc'].cast(DoubleType()))

In [50]:
crunched_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### เช็ค 'emp_length' หลัง Replace value

In [51]:
crunched_df.select(crunched_df['emp_length']).distinct().show()

+-----------+
| emp_length|
+-----------+
|    9 years|
|    5 years|
|     1 year|
|NotEmployed|
|    2 years|
|    7 years|
|    8 years|
|    4 years|
|    6 years|
|    3 years|
|  10+ years|
|   < 1 year|
+-----------+



In [52]:
crunched_df.select(crunched_df['purpose']).distinct().show()

+------------------+
|           purpose|
+------------------+
|             other|
|    small_business|
|debt_consolidation|
|       credit_card|
|            moving|
|          vacation|
|  renewable_energy|
|             house|
|               car|
|    major_purchase|
|           medical|
|  home_improvement|
+------------------+



### Normalization 'annual_inc'

In [53]:
from pyspark.sql.functions import *
max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]
min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [54]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [55]:
n_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

### Normalization 'loan_amnt'

In [56]:
#max_loan_amt = crunched_df.select(max('loan_amnt')).collect()[0][0]
#min_loan_amt = crunched_df.select(min('loan_amnt')).collect()[0][0]

In [57]:
#def t_loan_amt(origin):
    #return ((origin-min_loan_amt)/(max_loan_amt-min_loan_amt))

In [58]:
#n_loan_amt = udf(lambda x: t_loan_amt(x),DoubleType())

### Normalization 'revol_bal'

In [59]:
max_revol_bal = crunched_df.select(max('revol_bal')).collect()[0][0]
min_revol_bal = crunched_df.select(min('revol_bal')).collect()[0][0]

In [60]:
def t_revol_bal(origin):
    return ((origin-min_revol_bal)/(max_revol_bal-min_revol_bal))

In [61]:
n_revol_bal = udf(lambda x: t_revol_bal(x),DoubleType())

### Normalization 'tot_cur_bal'

In [62]:
max_tot_cur_bal = crunched_df.select(max('tot_cur_bal')).collect()[0][0]
min_tot_cur_bal = crunched_df.select(min('tot_cur_bal')).collect()[0][0]

In [63]:
def t_tot_cur_bal(origin):
    return ((origin-min_tot_cur_bal)/(max_tot_cur_bal-min_tot_cur_bal))

In [64]:
n_tot_cur_bal = udf(lambda x: t_tot_cur_bal(x),DoubleType())

### Normalization 'total_rev_hi_lim'

In [65]:
max_total_rev_hi_lim = crunched_df.select(max('total_rev_hi_lim')).collect()[0][0]
min_total_rev_hi_lim = crunched_df.select(min('total_rev_hi_lim')).collect()[0][0]

In [66]:
def t_total_rev_hi_lim(origin):
    return ((origin-min_total_rev_hi_lim)/(max_total_rev_hi_lim-min_total_rev_hi_lim))

In [67]:
n_total_rev_hi_lim = udf(lambda x: t_total_rev_hi_lim(x),DoubleType())

### Normalization 'total_bal_ex_mort'

In [68]:
max_total_bal_ex_mort = crunched_df.select(max('total_bal_ex_mort')).collect()[0][0]
min_total_bal_ex_mort = crunched_df.select(min('total_bal_ex_mort')).collect()[0][0]

In [69]:
def t_total_bal_ex_mort(origin):
    return ((origin-min_total_bal_ex_mort)/(max_total_bal_ex_mort-min_total_bal_ex_mort))

In [70]:
n_total_bal_ex_mort = udf(lambda x: t_total_bal_ex_mort(x),DoubleType())

### Normalization 'total_bal_il'

In [71]:
max_total_bal_il = crunched_df.select(max('total_bal_il')).collect()[0][0]
min_total_bal_il = crunched_df.select(min('total_bal_il')).collect()[0][0]

In [72]:
def t_total_bal_il(origin):
    return ((origin-min_total_bal_il)/(max_total_bal_il-min_total_bal_il))

In [73]:
n_total_bal_il = udf(lambda x: t_total_bal_il(x),DoubleType())

### รวมตัวแปรที่ทำการ Normalized แล้ว

In [74]:
normalized_df = crunched_df.\
withColumn('annual_inc',n_annual_inc(crunched_df['annual_inc'])).\
withColumn('revol_bal',n_revol_bal(crunched_df['revol_bal'])).\
withColumn('tot_cur_bal',n_tot_cur_bal(crunched_df['tot_cur_bal'])).\
withColumn('total_rev_hi_lim',n_total_rev_hi_lim(crunched_df['total_rev_hi_lim'])).\
withColumn('total_bal_il',n_total_bal_il(crunched_df['total_bal_il'])).\
withColumn('total_bal_ex_mort',n_total_bal_ex_mort(crunched_df['total_bal_ex_mort']))

#withColumn('loan_amnt',n_loan_amt(crunched_df['loan_amnt'])).\

In [75]:
#ดูข้อมูลที่มีการทำ Normalization (ให้ลองเปลี่ยน orderby เป็น GroupBy)

In [76]:
normalized_df.select(['grade'\
                      ,'annual_inc'\
                      ,'revol_bal'\
                      ,'tot_cur_bal'\
                      ,'revol_bal'\
                      ,'total_bal_il'\
                      ,'total_rev_hi_lim']).orderBy(['grade']).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|grade|          annual_inc|           revol_bal|         tot_cur_bal|           revol_bal|        total_bal_il|    total_rev_hi_lim|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    A|  0.0041759945029008|0.002972165172235302|0.006666284665671995|0.002972165172235302|0.023876915934627912| 0.01978570213730937|
|    A|0.015666586476262682| 0.03751325917860322| 0.06872142797848747| 0.03751325917860322|0.041307296691499164|0.029423436702760927|
|    A|0.015196516804625151| 0.02354004030263004|0.026245855840170782| 0.02354004030263004|0.012504987051800054| 0.06411928113838652|
|    A|0.003131395232595...|0.005018055332189111|0.003590993004239...|0.005018055332189111| 0.01004156480991172|0.027949430239809513|
|    A|0.008876691219276115|0.005140211145004223|0.01186627320

In [77]:
normalized_df.count()

627709

In [78]:
normalized_filtered_df = normalized_df\
#.filter(col('loan_status') == 'Fully Paid').\
#union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [79]:
normalized_filtered_df.count()

627709

In [80]:
#normalized_filtered_df.select('loan_status').distinct().show()

### สร้าง Dataframe โดยใช้ Pandas และให้แสดงข้อมูลหลัง Normalized

In [81]:
pd.DataFrame(normalized_df.describe().take(6),\
             columns=normalized_df.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,627709,0.008462566145329763,0.008274208621998926,0.0,1.0
bc_util,627709,55.70084688924327,28.831685669621613,0.0,252.3
inq_fi,627709,1.1189500230202212,1.585937864574576,0.0,48.0
inq_last_12m,627709,2.3041584555900902,2.5282249368970446,0.0,67.0
home_ownership,627709,None,None,ANY,RENT
purpose,627709,None,None,car,vacation
emp_length,627709,None,None,1 year,NotEmployed
revol_bal,627709,0.014265257680731206,0.019837633181117045,0.0,1.0
dti,627709,19.8488226232219,13.332388424437125,-1.0,999.0


### กำจัดค่า 'Null' จากการทำ Data Prep

In [82]:
data_no_missing_df = normalized_filtered_df.dropna(how='any')

In [83]:
data_no_missing_df.count()

627709

In [84]:
data_no_missing_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### 4) ML Modeling

In [85]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, \
OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, \
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, \
NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [86]:
#เปลี่ยนข้อมูลที่ต้องการ (Field) จาก Categorical เป็น Numerical

In [87]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [88]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',\
                                  outputCol='gradeVec')

In [89]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',\
                                  outputCol='homeVec')

In [90]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                  outputCol='purposeVec')

In [91]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                  outputCol='emp_lengthVec')

In [92]:
verification_statusIndexer = StringIndexer(inputCol='verification_status',outputCol='verification_statusIndexed')
verification_statusOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='verification_statusIndexed',\
                                  outputCol='verification_statusVec')

### เลือก Feature ที่จะใช้ในการ Train

In [93]:
featureAssembler = VectorAssembler(inputCols=['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'gradeVec'\
                                              ,'homeVec'\
                                              ,'emp_lengthVec'\
                                              ,'purposeVec'\
                                              ,'verification_statusVec']\
                                   ,outputCol='***features')

In [230]:
selected_attr_list = ['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'grade'\
                                              ,'home'\
                                              ,'emp_length'\
                                              ,'purpose'\
                                              ,'verification_status']

# Modeling with DecisionTree

In [94]:
dt = DecisionTreeClassifier(featuresCol='***features',labelCol='indexedLabel')

In [95]:
#ทำ Sequencing ลำดับของการทำ Modeling

In [96]:
pipeline_dt = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    dt])

### แบ่งข้อมูลเป็น Training และ Testing

In [97]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60% - 40%) โดย Random จากข้อมูลที่เหลืออยู่

In [98]:
training_dt, test_dt = normalized_df.randomSplit([0.6,0.4], seed = 13)

In [99]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data

In [100]:
training_dt.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78562|
| Fully Paid|297759|
+-----------+------+



In [101]:
training_dt.groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F|  7691|
|    E| 22169|
|    B|111691|
|    D| 52955|
|    C|113194|
|    A| 66392|
|    G|  2229|
+-----+------+



In [102]:
training_dt.count()

376321

In [103]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60%) โดย Random จาก Label ในแต่ละ Catergory 

In [104]:
#training, test = normalized_df\
#.filter(col('grade') == 'A').sample(False,0.4,42).\
#union(normalized_df.filter(col('grade') =='B').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='C').\
      #ample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='D').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='E')).\
#union(normalized_df.filter(col('grade') =='F')).\
#union(normalized_df.filter(col('grade') =='G')).\
#randomSplit([0.4,0.2], seed = 12)

In [105]:
start_time_dt = t.time()

In [106]:
model_dt = pipeline_dt.fit(training_dt)

In [107]:
duration_dt = t.time() - start_time_dt

In [108]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_dt))

ใช้เวลา Distributed Training 77.45955991744995 วินาที


In [109]:
spark_fullLabel_func = udf(lambda x: "Fully Paid" if x == 0.0 else "Charged Off")

In [110]:
result_dt = model_dt.transform(test_dt.dropna()).withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [111]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data (ข้อมูล Test)

In [112]:
result_dt.groupBy(['loan_status','indexedLabel']).count().show()

+-----------+------------+------+
|loan_status|indexedLabel| count|
+-----------+------------+------+
| Fully Paid|         0.0|198694|
|Charged Off|         1.0| 52694|
+-----------+------------+------+



In [113]:
result_dt.count()

251388

In [114]:
result_dt

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

In [115]:
num_result_dt = result_dt.count()

# การประเมิน Charge-Off [Aekanun]

In [116]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [117]:
num_Actual_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [118]:
num_Actual_ChargeOff

52694

In [119]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [120]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [121]:
num_False_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [122]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

19.499373742741106

In [123]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [124]:
num_True_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [125]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

80.50062625725889

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [126]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [127]:
num_Actual_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [128]:
num_Actual_FullyPaid

198694

In [129]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [130]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|  

In [131]:
num_False_ChargedOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [132]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.14192678188571373

In [133]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [134]:
num_True_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [135]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

99.85807321811428

# ------------- สิ้นสุดการประเมิน ---------------------------

In [136]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [137]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.958005155377345

In [138]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [139]:
evaluator_DT = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_DT.evaluate(result_dt)*100

95.8005155377345

# Modeling with RandomForest

In [140]:
RF = RandomForestClassifier(featuresCol='***features',labelCol='indexedLabel')

In [141]:
pipeline_RF = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    RF])

In [142]:
#training_RF, test_RF = normalized_df.randomSplit([0.6,0.4])

In [143]:
training_RF = training_dt

In [144]:
test_RF = test_dt

In [145]:
training_RF.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78562|
| Fully Paid|297759|
+-----------+------+



In [146]:
training_RF.count()

376321

In [147]:
start_time_rf = t.time()

In [148]:
model_RF = pipeline_RF.fit(training_RF)

In [149]:
duration_rf = t.time() - start_time_rf

In [150]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_rf))

ใช้เวลา Distributed Training 67.6216676235199 วินาที


In [151]:
result_RF = model_RF.transform(test_RF.dropna()).withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [152]:
result_RF.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|198694|
|Charged Off| 52694|
+-----------+------+



In [153]:
result_RF.count()

251388

In [154]:
result_RF

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [155]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [156]:
num_Actual_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [157]:
num_Actual_ChargeOff

52694

In [158]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [159]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [160]:
num_False_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [161]:
num_False_FullyPaid

44448

In [162]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

84.35115952480359

In [163]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [164]:
num_True_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [165]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

15.648840475196419

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [166]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [167]:
num_Actual_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [168]:
num_Actual_FullyPaid

198694

In [169]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [170]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [171]:
num_False_ChargedOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [172]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.0

In [173]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [174]:
num_True_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [175]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

100.0

# ------------- สิ้นสุดการประเมิน ---------------------------

In [176]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [177]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.8231896510573297

In [178]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [179]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_RF)*100

82.31896510573297

# Modeling with Multi layer Perceptron (ANN)

In [180]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [181]:
pipeline_mlp = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler])

In [182]:
#training_mlp, test_mlp = normalized_df.randomSplit([0.6,0.4])

In [183]:
training_mlp = training_dt

In [184]:
test_mlp = test_dt

In [185]:
train_df_features = pipeline_mlp.fit(training_mlp).transform(training_mlp)

In [186]:
layers = [train_df_features.schema["***features"].metadata["ml_attr"]["num_attrs"],20 ,10, 2]

In [187]:
layers

[43, 20, 10, 2]

### Implementation of MLP

In [188]:
clf = MultilayerPerceptronClassifier(labelCol='indexedLabel'\
                                    ,featuresCol='***features'\
                                    ,layers = layers)

In [189]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [190]:
pipeline_mlp_test = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,
                                    clf])

In [191]:
#training_mlp_test, test_mlp_test = normalized_df.randomSplit([0.6,0.4])

In [192]:
training_mlp.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78562|
| Fully Paid|297759|
+-----------+------+



In [193]:
training_mlp.count()

376321

In [194]:
start_time_mlp = t.time()

In [195]:
model_mlp = pipeline_mlp_test.fit(training_mlp)

In [196]:
duration_mlp = t.time() - start_time_mlp

In [197]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_mlp))

ใช้เวลา Distributed Training 46.91833758354187 วินาที


In [198]:
result_mlp = model_mlp.transform(test_mlp.dropna())\
.withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [199]:
result_mlp.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|198694|
|Charged Off| 52694|
+-----------+------+



In [200]:
result_mlp.count()

251388

In [201]:
result_mlp

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [202]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [203]:
num_Actual_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [204]:
num_Actual_ChargeOff

52694

In [205]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [206]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [207]:
num_False_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [208]:
num_False_FullyPaid

52694

In [209]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

100.0

In [210]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [211]:
num_True_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [212]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

0.0

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [213]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [214]:
num_Actual_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [215]:
num_Actual_FullyPaid

198694

In [216]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [217]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [218]:
num_False_ChargedOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [219]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.0

In [220]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [221]:
num_True_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [222]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

100.0

# ------------- สิ้นสุดการประเมิน ---------------------------

In [223]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [224]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.7903877671169666

In [225]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [226]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_mlp)*100

79.03877671169666

In [227]:
#model_mlp.save('./modelMLP_bestloanpayment')

In [228]:
! df -h

Filesystem                      Size  Used Avail Use% Mounted on
/dev/mapper/set1--vm1--vg-root   12G   11G  669M  94% /
none                            4.0K     0  4.0K   0% /sys/fs/cgroup
udev                            3.9G  4.0K  3.9G   1% /dev
tmpfs                           798M  840K  797M   1% /run
none                            5.0M     0  5.0M   0% /run/lock
none                            3.9G     0  3.9G   0% /run/shm
none                            100M   16K  100M   1% /run/user
/dev/sda1                       236M   37M  188M  17% /boot


In [229]:
! ls -l modelMLP_bestloanpayment

total 8
drwxr-xr-x  2 root root 4096 Oct 18 19:36 metadata
drwxr-xr-x 15 root root 4096 Oct 18 19:36 stages
